<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Download-and-Restore-Model-Weights" data-toc-modified-id="Download-and-Restore-Model-Weights-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Download and Restore Model Weights</a></span></li><li><span><a href="#Full-Model-Netron" data-toc-modified-id="Full-Model-Netron-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Full Model Netron</a></span></li><li><span><a href="#Split-Model" data-toc-modified-id="Split-Model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Split Model</a></span><ul class="toc-item"><li><span><a href="#Model-Part-1" data-toc-modified-id="Model-Part-1-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Model Part-1</a></span><ul class="toc-item"><li><span><a href="#Define" data-toc-modified-id="Define-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Define</a></span></li><li><span><a href="#Netron" data-toc-modified-id="Netron-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Netron</a></span><ul class="toc-item"><li><span><a href="#tf2onnx:-from_saved_model" data-toc-modified-id="tf2onnx:-from_saved_model-3.1.2.1"><span class="toc-item-num">3.1.2.1&nbsp;&nbsp;</span>tf2onnx: from_saved_model</a></span></li><li><span><a href="#tf2onnx:-from_keras" data-toc-modified-id="tf2onnx:-from_keras-3.1.2.2"><span class="toc-item-num">3.1.2.2&nbsp;&nbsp;</span>tf2onnx: from_keras</a></span></li></ul></li></ul></li><li><span><a href="#Model-Part-2" data-toc-modified-id="Model-Part-2-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Model Part-2</a></span><ul class="toc-item"><li><span><a href="#Define" data-toc-modified-id="Define-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Define</a></span></li><li><span><a href="#Netron" data-toc-modified-id="Netron-3.2.2"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>Netron</a></span><ul class="toc-item"><li><span><a href="#tf2onnx:-from_saved_model" data-toc-modified-id="tf2onnx:-from_saved_model-3.2.2.1"><span class="toc-item-num">3.2.2.1&nbsp;&nbsp;</span>tf2onnx: from_saved_model</a></span></li><li><span><a href="#tf2onnx:-from_keras" data-toc-modified-id="tf2onnx:-from_keras-3.2.2.2"><span class="toc-item-num">3.2.2.2&nbsp;&nbsp;</span>tf2onnx: from_keras</a></span></li></ul></li></ul></li><li><span><a href="#Model-Part-3" data-toc-modified-id="Model-Part-3-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Model Part-3</a></span><ul class="toc-item"><li><span><a href="#Define" data-toc-modified-id="Define-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>Define</a></span></li><li><span><a href="#Netron" data-toc-modified-id="Netron-3.3.2"><span class="toc-item-num">3.3.2&nbsp;&nbsp;</span>Netron</a></span><ul class="toc-item"><li><span><a href="#tf2onnx:-from_saved_model" data-toc-modified-id="tf2onnx:-from_saved_model-3.3.2.1"><span class="toc-item-num">3.3.2.1&nbsp;&nbsp;</span>tf2onnx: from_saved_model</a></span></li><li><span><a href="#tf2onnx:-from_keras" data-toc-modified-id="tf2onnx:-from_keras-3.3.2.2"><span class="toc-item-num">3.3.2.2&nbsp;&nbsp;</span>tf2onnx: from_keras</a></span></li></ul></li></ul></li><li><span><a href="#Test" data-toc-modified-id="Test-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Test</a></span></li></ul></li></ul></div>

In [88]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -p numpy,sklearn,pandas
%watermark -p ipywidgets,cv2,PIL,matplotlib,plotly,netron
%watermark -p torch,torchvision,torchaudio
%watermark -p tensorflow,tensorboard,tflite
%watermark -p onnx,tf2onnx,onnxruntime,tensorrt,tvm
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Image, Javascript
from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

import sys, os, io, time, random, math
import json, base64, requests, shutil
import os.path as osp
import numpy as np
import argparse, shlex, signal

argparse.ArgumentParser.exit = lambda *arg, **kwargs: _IGNORE_

def _IMPORT(x):
    try:
        x = x.strip()
        if x.startswith('https://'):
            x = x[8:]
        if not x.endswith('.py'):
            x = x + '.py'
        if x[0] == '/':
            with open(x) as fr:
                x = fr.read()
        else:
            x = x.replace('blob/main/', '').replace('blob/master/', '')
            if x.startswith('raw.githubusercontent.com'):
                uri = 'https://' + x
                x = requests.get(uri)
                if x.status_code == 200:
                    x = x.text
            elif x.startswith('github.com'):
                uri = x.replace('github.com', 'raw.githubusercontent.com')
                mod = uri.split('/')
                for s in ['main', 'master']:
                    uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[-3:])
                    x = requests.get(uri)
                    if x.status_code == 200:
                        x = x.text
                        break
            elif x.startswith('gitee.com'):
                mod = x.split('/')
                for s in ['/raw/main/', '/raw/master/']:
                    uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:])
                    x = requests.get(uri)
                    if x.status_code == 200:
                        x = x.text
                        break
        exec(x, globals())
    except:
        pass

def _DIR(x, dumps=True, ret=True):
    attrs = sorted([y for y in dir(x) if not y.startswith('_')])
    result = '%s: %s' % (str(type(x))[8:-2], json.dumps(attrs) if dumps else attrs)
    if ret:
        return result
    print(result)


numpy 1.19.5
sklearn 0.0
pandas 1.1.5
ipywidgets 7.6.3
cv2 4.5.3
PIL 8.3.1
matplotlib 3.3.4
plotly 5.3.0
netron 5.1.6
torch 1.8.1+cu101
torchvision 0.9.1+cu101
torchaudio not installed
tensorflow 2.6.0
tensorboard 2.6.0
tflite not installed
onnx 1.10.1
tf2onnx 1.10.0
onnxruntime 1.8.1
tensorrt not installed
tvm not installed


In [89]:
def display_html(port, height=600):
    from IPython import display
    from html import escape as html_escape
    frame_id = 'erlangai-frame-{:08x}'.format(random.getrandbits(64))
    shell = '''
      <iframe id='%HTML_ID%' width='100%' height='%HEIGHT%' frameborder='0'>
      </iframe>
      <script>
        (function() {
          const frame = document.getElementById(%JSON_ID%);
          const url = new URL(%URL%, window.location);
          const port = %PORT%;
          if (port) {
            url.port = port;
          }
          frame.src = url;
        })();
      </script>
    '''
    replacements = [
        ('%HTML_ID%', html_escape(frame_id, quote=True)),
        ('%JSON_ID%', json.dumps(frame_id)),
        ('%HEIGHT%', '%d' % height),
        ('%PORT%', '%d' % port),
        ('%URL%', json.dumps('/')),
    ]
    for (k, v) in replacements:
        shell = shell.replace(k, v)
    display.display(display.HTML(shell))

@register_line_magic
def netron(line):
    parser = argparse.ArgumentParser(prog='netron')
    parser.add_argument('--file', '-f', type=str, required=True, help='netron model file')
    parser.add_argument('--port', '-p', type=int, default=0, help='netron server port')
    parser.add_argument('--height', type=int, default=500, help='display netron html window hight')
    import netron
    try:
        args = parser.parse_args(shlex.split(line))
        address = netron.start(args.file, address=('0.0.0.0', args.port), browse=False)
        display_html(address[1], args.height)
    except:
        pass


In [90]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
import numpy as np
from tensorflow.python.saved_model.signature_constants import DEFAULT_SERVING_SIGNATURE_DEF_KEY
from tensorflow.python.saved_model.tag_constants import SERVING
import tf2onnx
import onnx
from onnxsim import simplify
import onnxruntime as ORT

In [91]:
_IMPORT('https://gitee.com/qrsforever/blog_source_codes/blob/master/AI/tensorflow/misc/ResnetPeriodEstimator.py')

In [92]:
BATCH_SIZE = 1 # get_sims_with_noloop if batch_size == 1 else get_sims(build trt will fail)
IMAGE_W = 112
IMAGE_H = IMAGE_W
NUM_FRAMES = 64
SIM_TEMPERATURE = 13.544

SAVED_MODEL_ROOT = '/data/nb_data/split_repnet_model'
PATH_TO_CKPT = '/data/pretrained/cv/repnet/'
INPUTS_NPY = f'{SAVED_MODEL_ROOT}/model_inputs_{BATCH_SIZE}_{IMAGE_H}_{NUM_FRAMES}.npy'
MODEL1_OUTPUTS = f'{SAVED_MODEL_ROOT}/model1_outputs_{BATCH_SIZE}_{IMAGE_H}_{NUM_FRAMES}.npy'
MODEL2_OUTPUTS = f'{SAVED_MODEL_ROOT}/model2_outputs_{BATCH_SIZE}_{IMAGE_H}_{NUM_FRAMES}.npy'
MODEL3_0_OUTPUTS = f'{SAVED_MODEL_ROOT}/model3_outputs_{BATCH_SIZE}_{IMAGE_H}_{NUM_FRAMES}_0.npy'
MODEL3_1_OUTPUTS = f'{SAVED_MODEL_ROOT}/model3_outputs_{BATCH_SIZE}_{IMAGE_H}_{NUM_FRAMES}_1.npy'
OUTPUTS_0_NPY = f'{SAVED_MODEL_ROOT}/model_outputs_{BATCH_SIZE}_{IMAGE_H}_{NUM_FRAMES}_0.npy'
OUTPUTS_1_NPY = f'{SAVED_MODEL_ROOT}/model_outputs_{BATCH_SIZE}_{IMAGE_H}_{NUM_FRAMES}_1.npy'

## Download and Restore Model Weights

In [93]:
!test ! -d $PATH_TO_CKPT && mkdir -p $PATH_TO_CKPT && \
    wget -P $PATH_TO_CKPT https://storage.googleapis.com/repnet_ckpt/checkpoint && \
    wget -P $PATH_TO_CKPT https://storage.googleapis.com/repnet_ckpt/ckpt-88.data-00000-of-00002 && \
    wget -P $PATH_TO_CKPT https://storage.googleapis.com/repnet_ckpt/ckpt-88.data-00001-of-00002 && \
    wget -P $PATH_TO_CKPT https://storage.googleapis.com/repnet_ckpt/ckpt-88.index

In [94]:
model = ResnetPeriodEstimator()
model.call = tf.function(model.call)
tf.train.Checkpoint(model=model).restore(f'{PATH_TO_CKPT}/ckpt-88').expect_partial()
if not os.path.exists(INPUTS_NPY):
    test_inputs = np.random.randn(BATCH_SIZE, NUM_FRAMES, IMAGE_H, IMAGE_W, 3).astype(np.float32)
    np.save(INPUTS_NPY, test_inputs)
else:
    test_inputs = np.load(INPUTS_NPY)
test_inputs_tensor = tf.convert_to_tensor(test_inputs)
INPUTS_NPY

'/data/nb_data/split_repnet_model/model_inputs_1_112_64.npy'

In [95]:
test_outputs = model(test_inputs_tensor)
if not os.path.exists(OUTPUTS_0_NPY):
    np.save(OUTPUTS_0_NPY, test_outputs[0])
    np.save(OUTPUTS_1_NPY, test_outputs[1])
else:
    test_outputs_0 = np.load(OUTPUTS_0_NPY)
    test_outputs_1 = np.load(OUTPUTS_1_NPY)
test_outputs[0].shape, test_outputs[1].shape

(TensorShape([1, 64, 32]), TensorShape([1, 64, 1]))

In [96]:
test_outputs_1.reshape(-1)[:3].tolist(), test_outputs[1].numpy().reshape(-1)[:3].tolist()

([1.2658531665802002, 1.3311336040496826, 1.4469563961029053],
 [1.2658531665802002, 1.3311336040496826, 1.4469563961029053])

In [97]:
test_outputs_0.reshape(-1)[:3].tolist(), test_outputs[0].numpy().reshape(-1)[:3].tolist()

([-12.783597946166992, 3.1049232482910156, 0.4515947699546814],
 [-12.783597946166992, 3.1049232482910156, 0.4515947699546814])

In [107]:
model.summary()

Model: "resnet_period_estimator_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_4 (Functional)         (None, None, None, 1024)  5209600   
_________________________________________________________________
conv3d_4 (Conv3D)            multiple                  14156288  
_________________________________________________________________
batch_normalization_4 (Batch multiple                  2048      
_________________________________________________________________
conv2d_4 (Conv2D)            multiple                  320       
_________________________________________________________________
dense_92 (Dense)             multiple                  1049088   
_________________________________________________________________
dense_93 (Dense)             multiple                  1049088   
_________________________________________________________________
transformer_layer_8 (Transfo multiple    

## Full Model Netron

In [105]:
# TODO
if 1 == model.pos_encoding.shape[-1]:
    model.pos_encoding  = tf.tile(model.pos_encoding,  multiples=[1, 1, 512])
    model.pos_encoding2 = tf.tile(model.pos_encoding2, multiples=[1, 1, 512])

In [106]:
model_proto = tf2onnx.convert.from_keras(
    model, opset=11, 
    input_signature=(tf.TensorSpec(shape=test_inputs.shape, name="x"),),
    output_path=f'{SAVED_MODEL_ROOT}/model.onnx')[0];

KeyboardInterrupt: 

In [101]:
model_proto.graph.input, model_proto.graph.output

([name: "x"
 type {
   tensor_type {
     elem_type: 1
     shape {
       dim {
         dim_value: 1
       }
       dim {
         dim_value: 64
       }
       dim {
         dim_value: 112
       }
       dim {
         dim_value: 112
       }
       dim {
         dim_value: 3
       }
     }
   }
 }
 ],
 [name: "output_1"
 type {
   tensor_type {
     elem_type: 1
     shape {
       dim {
         dim_value: 1
       }
       dim {
         dim_value: 64
       }
       dim {
         dim_value: 32
       }
     }
   }
 }
 , name: "output_2"
 type {
   tensor_type {
     elem_type: 1
     shape {
       dim {
         dim_value: 1
       }
       dim {
         dim_value: 64
       }
       dim {
         dim_value: 1
       }
     }
   }
 }
 ])

In [102]:
# %netron -f {SAVED_MODEL_ROOT}/model.onnx

## Split Model

### Model Part-1

#### Define

In [108]:
class ResnetPart1(tf.keras.models.Model):
    def __init__(self, model):
        super().__init__(name='ResnetPart1')
        self.base_model = tf.keras.models.Model(
            name='base_model', inputs=model.base_model.input,
            outputs=model.base_model.get_layer('conv4_block3_out').output)
    
    def call(self, x):
        x = tf.reshape(x, (-1, IMAGE_H, IMAGE_W, 3))
        return self.base_model(x)

In [109]:
model1 = ResnetPart1(model)
model_part1_outputs = model1(test_inputs_tensor)
tf.reshape(model_part1_outputs, (-1,))[:3], '-'*90, model_part1_outputs.shape

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 0.6067329 , -0.3791368 , -0.15414873], dtype=float32)>,
 '------------------------------------------------------------------------------------------',
 TensorShape([64, 7, 7, 1024]))

In [110]:
model1.summary(line_length=90)

Model: "ResnetPart1"
__________________________________________________________________________________________
Layer (type)                            Output Shape                        Param #       
base_model (Functional)                 (None, None, None, 1024)            5209600       
Total params: 5,209,600
Trainable params: 5,193,856
Non-trainable params: 15,744
__________________________________________________________________________________________


In [111]:
np.save(MODEL1_OUTPUTS, model_part1_outputs)

#### Netron

##### tf2onnx: from_saved_model

In [112]:
model1.save(filepath=f'{SAVED_MODEL_ROOT}/model1', save_format='tf')

INFO:tensorflow:Assets written to: /data/nb_data/split_repnet_model/model1/assets


/usr/local/lib/python3.6/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning:

Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.



In [113]:
!python3 -m tf2onnx.convert --opset 11 \
    --tag {SERVING} --signature_def {DEFAULT_SERVING_SIGNATURE_DEF_KEY} \
    --saved-model {SAVED_MODEL_ROOT}/model1 --output {SAVED_MODEL_ROOT}/model1/model_opset11.onnx 

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2021-11-02 12:02:54,606 - INFO - Signatures found in model: [serving_default].
2021-11-02 12:02:54,608 - INFO - Output names: ['output_1']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-11-02 12:02:57,619 - WARNING - From /usr/local/lib/python3.6/dist-packages/tf2onnx/tf_loader.py:706: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-11-02 12:02:58,841 - INFO - Using tensorflow=2.6.0, onnx=1.10.1, tf2onnx=1.10.0/42e800
2021-11-02 12:02:58,847 - INFO - Using opset <onnx, 11>
2021-11-02 12:03:04,565 - INFO - Computed 0 values for constant folding
2021-11-02 12:03:08,58

In [114]:
# %netron -f {SAVED_MODEL_ROOT}/model1/model_opset11.onnx --height 360

##### tf2onnx: from_keras

In [115]:
model1_proto = tf2onnx.convert.from_keras(
    model1, opset=11, 
    input_signature=(tf.TensorSpec(shape=test_inputs_tensor.shape, name="model1_x"),),
    output_path=f'{SAVED_MODEL_ROOT}/model1.onnx')[0];

In [116]:
model1_proto.graph.input, model1_proto.graph.output

([name: "model1_x"
 type {
   tensor_type {
     elem_type: 1
     shape {
       dim {
         dim_value: 1
       }
       dim {
         dim_value: 64
       }
       dim {
         dim_value: 112
       }
       dim {
         dim_value: 112
       }
       dim {
         dim_value: 3
       }
     }
   }
 }
 ],
 [name: "output_1"
 type {
   tensor_type {
     elem_type: 1
     shape {
       dim {
         dim_value: 64
       }
       dim {
         dim_value: 7
       }
       dim {
         dim_value: 7
       }
       dim {
         dim_value: 1024
       }
     }
   }
 }
 ])

In [65]:
# %netron -f {SAVED_MODEL_ROOT}/model1.onnx --height 360

### Model Part-2

#### Define

In [53]:
class ResnetPart2(tf.keras.models.Model):
    def __init__(self, model):
        super().__init__(name='ResnetPart2')
        self.temporal_conv_layer = layers.Conv3D(
            512, 3, padding='same', name='temporal_conv_layer',
            dilation_rate=(3, 1, 1), weights=model.temporal_conv_layers[0].get_weights())
        self.temporal_bn_layers = layers.BatchNormalization(
            name='temporal_bn_layers', weights=model.temporal_bn_layers[0].get_weights())                  
        
    def call(self, x):
        x = tf.reshape(x, [BATCH_SIZE, -1] + x.shape.as_list()[1:])
        x = self.temporal_conv_layer(x)
        x = self.temporal_bn_layers(x)
        x = tf.nn.relu(x)
        return tf.reduce_max(x, [2, 3])

In [54]:
model2 = ResnetPart2(model)
model_part2_outputs = model2(model_part1_outputs)
tf.reshape(model_part2_outputs, (-1,))[:3]

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.       , 2.9224107, 0.       ], dtype=float32)>

In [99]:
model2.summary(line_length=90)

Model: "ResnetPart2"
__________________________________________________________________________________________
Layer (type)                            Output Shape                        Param #       
temporal_conv_layer (Conv3D)            multiple                            14156288      
__________________________________________________________________________________________
temporal_bn_layers (BatchNormalization) multiple                            2048          
Total params: 14,158,336
Trainable params: 14,157,312
Non-trainable params: 1,024
__________________________________________________________________________________________


In [69]:
np.save(MODEL2_OUTPUTS, model_part2_outputs)

#### Netron

##### tf2onnx: from_saved_model

In [70]:
model2.save(filepath=f'{SAVED_MODEL_ROOT}/model2', save_format='tf')

INFO:tensorflow:Assets written to: /data/nb_data/split_repnet_model/model2/assets


INFO:tensorflow:Assets written to: /data/nb_data/split_repnet_model/model2/assets


In [30]:
!python3 -m tf2onnx.convert --opset 11 \
    --tag {SERVING} --signature_def {DEFAULT_SERVING_SIGNATURE_DEF_KEY} \
    --saved-model {SAVED_MODEL_ROOT}/model2 --output {SAVED_MODEL_ROOT}/model2/model_opset11.onnx 

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2021-10-21 19:45:12,444 - INFO - Signatures found in model: [serving_default].
2021-10-21 19:45:12,444 - INFO - Output names: ['output_1']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-10-21 19:45:14,465 - WARNING - From /usr/local/lib/python3.6/dist-packages/tf2onnx/tf_loader.py:706: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-10-21 19:45:15,998 - INFO - Using tensorflow=2.6.0, onnx=1.10.1, tf2onnx=1.10.0/42e800
2021-10-21 19:45:15,998 - INFO - Using opset <onnx, 11>
2021-10-21 19:45:23,516 - INFO - Computed 2 values for constant folding
2021-10-21 19:45:27,83

In [31]:
%netron -f {SAVED_MODEL_ROOT}/model2/model_opset11.onnx --height 360

Serving '/data/nb_data/split_repnet_model/model2/model_opset11.onnx' at http://0.0.0.0:38712


##### tf2onnx: from_keras

In [128]:
model2_proto = tf2onnx.convert.from_keras(
    model2, opset=11, 
    input_signature=(tf.TensorSpec(shape=model_part1_outputs.shape, name="model2_x"),),
    output_path=f'{SAVED_MODEL_ROOT}/model2.onnx')[0];

In [33]:
model2_proto.graph.input, model2_proto.graph.output

([name: "model2_x"
 type {
   tensor_type {
     elem_type: 1
     shape {
       dim {
         dim_value: 64
       }
       dim {
         dim_value: 7
       }
       dim {
         dim_value: 7
       }
       dim {
         dim_value: 1024
       }
     }
   }
 }
 ],
 [name: "output_1"
 type {
   tensor_type {
     elem_type: 1
     shape {
       dim {
         dim_value: 1
       }
       dim {
         dim_value: 64
       }
       dim {
         dim_value: 512
       }
     }
   }
 }
 ])

In [34]:
%netron -f {SAVED_MODEL_ROOT}/model2.onnx --height 360

Serving '/data/nb_data/split_repnet_model/model2.onnx' at http://0.0.0.0:42016


### Model Part-3

#### Define

In [55]:
def get_sims_with_noloop(embs, temperature):
    """Calculates self-similarity between batch of sequence of embeddings."""
    # batch_size = tf.shape(embs)[0]
    # seq_len = tf.shape(embs)[1]
    # embs = tf.reshape(embs, [batch_size, seq_len, -1])

    def _get_sims(embs):
        """Calculates self-similarity between sequence of embeddings."""
        dist = pairwise_l2_distance(embs, embs)
        sims = -1.0 * dist
        return sims
    
    # batchsize = 4
    # a = tf.expand_dims(_get_sims(embs[0]), 0)
    # b = tf.expand_dims(_get_sims(embs[1]), 0)
    # c = tf.expand_dims(_get_sims(embs[2]), 0)
    # d = tf.expand_dims(_get_sims(embs[3]), 0)
    # sims = tf.concat([a, b, c, d], 0)
    sims = tf.expand_dims(_get_sims(embs[0]), 0)
    sims /= temperature
    sims = tf.nn.softmax(sims, axis=-1)
    sims = tf.expand_dims(sims, -1)
    return sims


class ResnetPart3(tf.keras.models.Model):
    def __init__(self, model):
        super().__init__(name='ResnetPart3')
        self.conv_3x3_layer = layers.Conv2D(
            32, 3, padding='same', activation=tf.nn.relu, name='conv_3x3_layer',
            weights=model.conv_3x3_layer.get_weights())
        
        self.input_projection1 = layers.Dense(
            512, activation=None, name='input_projection1',
            weights=model.input_projection.get_weights())
        self.input_projection2 = layers.Dense(
            512, activation=None, name='input_projection2',
            weights=model.input_projection2.get_weights())
        
        self.pos_encoding1 = tf.compat.v1.get_variable(name='pos_encoding',  initializer=model.pos_encoding.numpy())
        self.pos_encoding2 = tf.compat.v1.get_variable(name='pos_encoding2', initializer=model.pos_encoding2.numpy())
        
        self.transformer_layer1 = TransformerLayer(512, 4, 512, 0.0, True, name='transformer_layer1')
        self.transformer_layer1(tf.random.uniform((BATCH_SIZE, NUM_FRAMES, 512)))
        self.transformer_layer1.set_weights(model.transformer_layers[0].get_weights())
        
        self.transformer_layer2 = TransformerLayer(512, 4, 512, 0.0, True, name='transformer_layer2')
        self.transformer_layer2(tf.random.uniform((BATCH_SIZE, NUM_FRAMES, 512)))
        self.transformer_layer2.set_weights(model.transformer_layers2[0].get_weights())
        
        self.dropout_layer = layers.Dropout(0.25, name='dropout', weights=model.dropout_layer.get_weights())
        self.fc_layers = [
            layers.Dense(512, activation=tf.nn.relu, name='fc_0', weights=model.fc_layers[0].get_weights()),
            layers.Dense(512, activation=tf.nn.relu, name='fc_1', weights=model.fc_layers[1].get_weights()),
            layers.Dense(NUM_FRAMES//2, name='fc_layers_2', weights=model.fc_layers[2].get_weights()),
        ]
        
        self.within_period_fc_layers = [
            layers.Dense(512, activation=tf.nn.relu, name='within_period_fc_0', weights=model.within_period_fc_layers[0].get_weights()),
            layers.Dense(512, activation=tf.nn.relu, name='within_period_fc_1', weights=model.within_period_fc_layers[1].get_weights()),
            layers.Dense(1, name='within_period_fc_2', weights=model.within_period_fc_layers[2].get_weights()),
        ]
        
        # TODO ValueError: Dimensions must be equal, but are 1 and 512 for Add
        
        if self.pos_encoding1.shape[-1] == 1:
            self.pos_encoding1 = tf.tile(self.pos_encoding1, multiples=[1, 1, 512])
            self.pos_encoding2 = tf.tile(self.pos_encoding2, multiples=[1, 1, 512])
                                            
    def call(self, x):
        x = get_sims_with_noloop(x, SIM_TEMPERATURE)
        x = self.conv_3x3_layer(x)
        x = tf.reshape(x, [BATCH_SIZE, NUM_FRAMES, -1])
        within_period_x = x
        
        x = self.input_projection1(x)
        x += self.pos_encoding1
        x = self.transformer_layer1(x)
        x = flatten_sequential_feats(x, BATCH_SIZE, NUM_FRAMES)
        for fc_layer in self.fc_layers:
            x = self.dropout_layer(x)
            x = fc_layer(x)
            
        within_period_x = self.input_projection2(within_period_x)
        within_period_x += self.pos_encoding2
        within_period_x = self.transformer_layer2(within_period_x)
        within_period_x = flatten_sequential_feats(within_period_x, BATCH_SIZE, NUM_FRAMES)
        for fc_layer in self.within_period_fc_layers:
            within_period_x = self.dropout_layer(within_period_x)
            within_period_x = fc_layer(within_period_x)
          
        return x, within_period_x

In [56]:
model3 = ResnetPart3(model)
model_part3_outputs = model3(model_part2_outputs)
tf.reshape(model_part3_outputs[0], (-1,))[:3], '-'*60, tf.reshape(test_outputs[0], (-1,))[:3]

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-13.967211 ,   0.3748341,  -0.9406101], dtype=float32)>,
 '------------------------------------------------------------',
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-12.783598  ,   3.1049232 ,   0.45159477], dtype=float32)>)

In [37]:
model3.summary(line_length=90)

Model: "ResnetPart3"
__________________________________________________________________________________________
Layer (type)                            Output Shape                        Param #       
conv_3x3_layer (Conv2D)                 multiple                            320           
__________________________________________________________________________________________
input_projection1 (Dense)               multiple                            1049088       
__________________________________________________________________________________________
input_projection2 (Dense)               multiple                            1049088       
__________________________________________________________________________________________
transformer_layer1 (TransformerLayer)   multiple                            1577984       
__________________________________________________________________________________________
transformer_layer2 (TransformerLayer)   multiple                     

In [131]:
np.save(MODEL3_0_OUTPUTS, model_part3_outputs[0])
np.save(MODEL3_1_OUTPUTS, model_part3_outputs[1])

#### Netron

##### tf2onnx: from_saved_model

In [39]:
model3.save(filepath=f'{SAVED_MODEL_ROOT}/model3', save_format='tf')

INFO:tensorflow:Assets written to: /data/nb_data/split_repnet_model/model3/assets


INFO:tensorflow:Assets written to: /data/nb_data/split_repnet_model/model3/assets


In [40]:
!python3 -m tf2onnx.convert --opset 11 \
    --tag {SERVING} --signature_def {DEFAULT_SERVING_SIGNATURE_DEF_KEY} \
    --saved-model {SAVED_MODEL_ROOT}/model3 --output {SAVED_MODEL_ROOT}/model3/model_opset11.onnx 

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2021-10-21 19:46:00,960 - INFO - Signatures found in model: [serving_default].
2021-10-21 19:46:00,961 - INFO - Output names: ['output_1', 'output_2']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-10-21 19:46:02,217 - WARNING - From /usr/local/lib/python3.6/dist-packages/tf2onnx/tf_loader.py:706: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-10-21 19:46:02,690 - INFO - Using tensorflow=2.6.0, onnx=1.10.1, tf2onnx=1.10.0/42e800
2021-10-21 19:46:02,690 - INFO - Using opset <onnx, 11>
2021-10-21 19:46:06,042 - INFO - Computed 0 values for constant folding
2021-10-21

In [41]:
# %netron -f {SAVED_MODEL_ROOT}/model3/model_opset11.onnx --height 360

##### tf2onnx: from_keras

In [42]:
model3_proto = tf2onnx.convert.from_keras(
    model3, opset=11, 
    input_signature=(tf.TensorSpec(shape=model_part2_outputs.shape, name="model3_x"),),
    output_path=f'{SAVED_MODEL_ROOT}/model3.onnx')[0];

In [43]:
model3_proto.graph.input, model3_proto.graph.output

([name: "model3_x"
 type {
   tensor_type {
     elem_type: 1
     shape {
       dim {
         dim_value: 1
       }
       dim {
         dim_value: 64
       }
       dim {
         dim_value: 512
       }
     }
   }
 }
 ],
 [name: "output_1"
 type {
   tensor_type {
     elem_type: 1
     shape {
       dim {
         dim_value: 1
       }
       dim {
         dim_value: 64
       }
       dim {
         dim_value: 32
       }
     }
   }
 }
 , name: "output_2"
 type {
   tensor_type {
     elem_type: 1
     shape {
       dim {
         dim_value: 1
       }
       dim {
         dim_value: 64
       }
       dim {
         dim_value: 1
       }
     }
   }
 }
 ])

In [44]:
# %netron -f {SAVED_MODEL_ROOT}/model3.onnx --height 360

### Test

In [102]:
outputs = model3(model2(model1(test_inputs_tensor)))
outputs[0].shape, outputs[1].shape

(TensorShape([2, 64, 32]), TensorShape([2, 64, 1]))

In [103]:
tf.reshape(outputs[0], (-1,))[:3], '-'*60, tf.reshape(test_outputs[0], (-1,))[:3]

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-13.291716 ,   0.5982778,  -1.0268755], dtype=float32)>,
 '------------------------------------------------------------',
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-13.29172  ,   0.5982779,  -1.0268753], dtype=float32)>)

In [47]:
tf.reshape(outputs[1], (-1,))[:3], '-'*60, tf.reshape(test_outputs[1], (-1,))[:3]

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([1.2658517, 1.3311325, 1.4469559], dtype=float32)>,
 '------------------------------------------------------------',
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([1.2658532, 1.3311336, 1.4469564], dtype=float32)>)

In [132]:
class FullModel(tf.keras.models.Model):
    def __init__(self, model):
        super().__init__(name='Resnet')
        self.model1 = ResnetPart1(model)
        self.model2 = ResnetPart2(model)
        self.model3 = ResnetPart3(model)
        
    def call(self, x):
        return self.model3(self.model2(self.model1(x)))

In [133]:
full_model = FullModel(model)

In [134]:
full_outputs = full_model(test_inputs_tensor)

In [135]:
tf.reshape(full_outputs[0], (-1,))[:3], '-'*60, tf.reshape(test_outputs[0], (-1,))[:3]

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-11.92954   ,   1.6399554 ,  -0.06519073], dtype=float32)>,
 '------------------------------------------------------------',
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-11.929536  ,   1.6399661 ,  -0.06518548], dtype=float32)>)

In [136]:
full_model_proto = tf2onnx.convert.from_keras(
    full_model, opset=11,
    input_signature=(tf.TensorSpec(shape=test_inputs.shape, name="x"),),
    output_path=f'{SAVED_MODEL_ROOT}/full_model.onnx')[0];
# full_model_proto.graph.input, full_model_proto.graph.output

In [53]:
%netron -f {SAVED_MODEL_ROOT}/full_model.onnx

Serving '/data/nb_data/split_repnet_model/full_model.onnx' at http://0.0.0.0:33981


<div style="margin-top:30px; width: 100%;">
    <div style="text-align:right">
        <a
           title="Back to Top"
           href="#Contents"
           onclick="window.scrollTo(0, 0);"
           style="color:blue;font-size:28px;text-decoration:none;">
               &#128285;
        </a>
    </div>
</div>